In [ ]:
# Install required Packages
!pip install -q huggingface_hub==0.25.0 diffusers transformers accelerate ftfy torch torchvision
!pip install -q ffmpeg-python streamlit

In [ ]:
# Load dependencies
import os
import torch
from diffusers import StableDiffusionPipeline
import ffmpeg
from PIL import Image

In [ ]:
# Create working folders
os.makedirs("data/generated_videos/temp", exist_ok=True)

In [ ]:
# Load the Pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    revision="fp16" if torch.cuda.is_available() else None,
    low_cpu_mem_usage = True
).to(device)

In [ ]:
# generate frames
def generate_sequence(prompt: str, num_frames: str, int = 8, output_dir: str = "data/generated_videos/temp"):
  images = []
  for i in range (num_frames):
    image = pipe(f"{prompt}, frame {i + 1}", guidance_scale=7.5).images[0]
    path = os.path.join(output_dir, f"frame_{i:03}.png")
    image.save(path)
    images.append(path)
  return images

In [ ]:
# Create a video from frames
def create_video(frames_dir="data/generated_videos/temp", output_path="output.mp4", fps=2):
  (
      ffmpeg
      .input(os.path.join(frames_dir, 'frame_%03d.png'), framerate=fps)
      .output(output_path, vcodec='libx264', pix_fmt='yuv420p')
      .overwrite_output()
      .run()
  )

  return output_path

In [ ]:
# Run a sample prompt
frames = generate_sequence("a fire on a factory floor", num_frames=6)
video_path = create_video()

In [ ]:
from IPython.display import Video
Video(video_path)